In [2]:
from prometheus_api_client import PrometheusConnect, MetricsList
from prometheus_api_client.metric_range_df import MetricRangeDataFrame
from prometheus_api_client.utils import parse_datetime
import pandas as pd
import json
import pickle

The purpose of this notebook is to explore prometheus_api_client behaviour when extracting metric data, and accordingly decide what will be the "right" behaviour of query_to_df() in utils and the affected architecture of the project, specifically decide if it allowed to get as input a metric or query that result in multiple time series, such as summary metric. in such cases custom_query_range will result in sequences of jsons where each json is reffered to unique combination of labels (for example, unique combination of job, instance and quantile).

In [2]:
prom = PrometheusConnect(url="http://localhost:9090", disable_ssl=True)

define function to extract query range and format it as dataframe:

In [3]:
def query2df(query, columns=None):
    query_range = prom.custom_query_range(query ,
                            start_time=start_time,
                            end_time=end_time,
                            step=step)
    return MetricRangeDataFrame(query_range,columns=columns)

lets choose a summary metric:

In [4]:
start_time = parse_datetime("5min")
end_time = parse_datetime("now")
step = "5s"

query = "prometheus_target_interval_length_seconds"

df = query2df(query)
df.head()

,__name__,instance,interval,job,quantile,value
timestamp,,,,,,
1639132046,prometheus_target_interval_length_seconds,localhost:9090,15s,prometheus,0.01,14.9851886
1639132051,prometheus_target_interval_length_seconds,localhost:9090,15s,prometheus,0.01,14.9851886
1639132056,prometheus_target_interval_length_seconds,localhost:9090,15s,prometheus,0.01,14.9851886
1639132061,prometheus_target_interval_length_seconds,localhost:9090,15s,prometheus,0.01,14.9851886
1639132066,prometheus_target_interval_length_seconds,localhost:9090,15s,prometheus,0.01,14.9851886


In [5]:
df.describe()

,__name__,instance,interval,job,quantile,value
count,305,305,305,305,305,305
unique,1,1,1,1,5,15
top,prometheus_target_interval_length_seconds,localhost:9090,15s,prometheus,0.01,14.9874618
freq,305,305,305,305,61,61


we can see that the query results in additional columns due to the different quantiles. lets find all the unique combination of labels columns:

In [6]:
list(df.columns[1:-1])

['instance', 'interval', 'job', 'quantile']

In [7]:
groups = df.groupby(list(df.columns[1:-1]))
groups.groups.keys()

dict_keys([('localhost:9090', '15s', 'prometheus', '0.01'), ('localhost:9090', '15s', 'prometheus', '0.05'), ('localhost:9090', '15s', 'prometheus', '0.5'), ('localhost:9090', '15s', 'prometheus', '0.9'), ('localhost:9090', '15s', 'prometheus', '0.99')])

As expected there are 5 distinct metrics, against the 5 quantiles in summary metrics. In general it possible that there will be more metrics due to different instances or other labels.

Ofcourse we can define unique query that gives only one metric:

In [8]:
query = 'prometheus_target_interval_length_seconds{instance="localhost:9090", job="prometheus", quantile="0.01"}'

df = query2df(query)
df.head()


,__name__,instance,interval,job,quantile,value
timestamp,,,,,,
1639132046,prometheus_target_interval_length_seconds,localhost:9090,15s,prometheus,0.01,14.9851886
1639132051,prometheus_target_interval_length_seconds,localhost:9090,15s,prometheus,0.01,14.9851886
1639132056,prometheus_target_interval_length_seconds,localhost:9090,15s,prometheus,0.01,14.9851886
1639132061,prometheus_target_interval_length_seconds,localhost:9090,15s,prometheus,0.01,14.9851886
1639132066,prometheus_target_interval_length_seconds,localhost:9090,15s,prometheus,0.01,14.9851886


In [9]:
df.describe()

,__name__,instance,interval,job,quantile,value
count,61,61,61,61,61,61
unique,1,1,1,1,1,2
top,prometheus_target_interval_length_seconds,localhost:9090,15s,prometheus,0.01,14.9851886
freq,61,61,61,61,61,46


In [10]:
groups = df.groupby(['instance', 'job', 'quantile'])
groups.groups.keys()

dict_keys([('localhost:9090', 'prometheus', '0.01')])

Note, that if we pass specific columns to custom_query_range() we still get all the 305 values but without the the quantile label. No aggregation is done:

In [11]:
df = query2df("prometheus_target_interval_length_seconds", columns=['timestamp','instance','value'])
df.head()

,instance,value
timestamp,,
1639132046,localhost:9090,14.9851886
1639132051,localhost:9090,14.9851886
1639132056,localhost:9090,14.9851886
1639132061,localhost:9090,14.9851886
1639132066,localhost:9090,14.9851886


In [12]:
df.describe()

,instance,value
count,305,305
unique,1,15
top,localhost:9090,14.9874618
freq,305,61


lets load again the summary metric to explore the manipulation need to be done to get the desired output to be fitted with prophet:

In [13]:
query = "prometheus_target_interval_length_seconds"
df = query2df(query)

In [14]:
df = df.reset_index().rename(columns={'value':'y', 'timestamp':'ds'}).astype({'y':'float'})
df.head()

,ds,__name__,instance,interval,job,quantile,y
0,1639132046,prometheus_target_interval_length_seconds,localhost:9090,15s,prometheus,0.01,14.985189
1,1639132051,prometheus_target_interval_length_seconds,localhost:9090,15s,prometheus,0.01,14.985189
2,1639132056,prometheus_target_interval_length_seconds,localhost:9090,15s,prometheus,0.01,14.985189
3,1639132061,prometheus_target_interval_length_seconds,localhost:9090,15s,prometheus,0.01,14.985189
4,1639132066,prometheus_target_interval_length_seconds,localhost:9090,15s,prometheus,0.01,14.985189


In [15]:
df.dtypes

ds            int64
__name__     object
instance     object
interval     object
job          object
quantile     object
y           float64
dtype: object

In [16]:
df['ds'] = pd.to_datetime(df['ds'],unit='s').astype('datetime64[ns, Asia/Jerusalem]').dt.tz_localize(None)
df.head()

,ds,__name__,instance,interval,job,quantile,y
0,2021-12-10 12:27:26,prometheus_target_interval_length_seconds,localhost:9090,15s,prometheus,0.01,14.985189
1,2021-12-10 12:27:31,prometheus_target_interval_length_seconds,localhost:9090,15s,prometheus,0.01,14.985189
2,2021-12-10 12:27:36,prometheus_target_interval_length_seconds,localhost:9090,15s,prometheus,0.01,14.985189
3,2021-12-10 12:27:41,prometheus_target_interval_length_seconds,localhost:9090,15s,prometheus,0.01,14.985189
4,2021-12-10 12:27:46,prometheus_target_interval_length_seconds,localhost:9090,15s,prometheus,0.01,14.985189


so these are the identical manipulation as we do in query_to_df(). let see how we get each group:

In [17]:
dfgb = df.groupby(list(df.columns[1:-1]))

for example the first group (quantile 0.01):

In [18]:
dfgb.get_group(list(dfgb.groups.keys())[0])

,ds,__name__,instance,interval,job,quantile,y
0,2021-12-10 12:27:26,prometheus_target_interval_length_seconds,localhost:9090,15s,prometheus,0.01,14.985189
1,2021-12-10 12:27:31,prometheus_target_interval_length_seconds,localhost:9090,15s,prometheus,0.01,14.985189
2,2021-12-10 12:27:36,prometheus_target_interval_length_seconds,localhost:9090,15s,prometheus,0.01,14.985189
3,2021-12-10 12:27:41,prometheus_target_interval_length_seconds,localhost:9090,15s,prometheus,0.01,14.985189
4,2021-12-10 12:27:46,prometheus_target_interval_length_seconds,localhost:9090,15s,prometheus,0.01,14.985189
...,...,...,...,...,...,...,...
56,2021-12-10 12:32:06,prometheus_target_interval_length_seconds,localhost:9090,15s,prometheus,0.01,14.987462
57,2021-12-10 12:32:11,prometheus_target_interval_length_seconds,localhost:9090,15s,prometheus,0.01,14.987462
58,2021-12-10 12:32:16,prometheus_target_interval_length_seconds,localhost:9090,15s,prometheus,0.01,14.987462
59,2021-12-10 12:32:21,prometheus_target_interval_length_seconds,localhost:9090,15s,prometheus,0.01,14.987462


but maybe there is more efficient way to extract the data. lets explore MetricList:

In [19]:
query_range = prom.custom_query_range(query ,
                            start_time=start_time,
                            end_time=end_time,
                            step=step)

metric_object_list = MetricsList(query_range)

In [20]:
metric_object_list

In [21]:
for item in metric_object_list:
    print(item.metric_name, "\n")

prometheus_target_interval_length_seconds 

prometheus_target_interval_length_seconds 

prometheus_target_interval_length_seconds 

prometheus_target_interval_length_seconds 

prometheus_target_interval_length_seconds 



In [22]:
for item in metric_object_list:
    print(item.metric_name, item.label_config, "\n")

prometheus_target_interval_length_seconds {'instance': 'localhost:9090', 'interval': '15s', 'job': 'prometheus', 'quantile': '0.01'} 

prometheus_target_interval_length_seconds {'instance': 'localhost:9090', 'interval': '15s', 'job': 'prometheus', 'quantile': '0.05'} 

prometheus_target_interval_length_seconds {'instance': 'localhost:9090', 'interval': '15s', 'job': 'prometheus', 'quantile': '0.5'} 

prometheus_target_interval_length_seconds {'instance': 'localhost:9090', 'interval': '15s', 'job': 'prometheus', 'quantile': '0.9'} 

prometheus_target_interval_length_seconds {'instance': 'localhost:9090', 'interval': '15s', 'job': 'prometheus', 'quantile': '0.99'} 



so we can see that MetricList save the name and the labels of the metric. but the big thing is that it saves also the metric values in the exact format that we need in Prophet:

In [23]:
for item in metric_object_list:
    print(item)

{
metric_name: 'prometheus_target_interval_length_seconds'
label_config: {'instance': 'localhost:9090', 'interval': '15s', 'job': 'prometheus', 'quantile': '0.01'}
metric_values:                     ds          y
0  2021-12-10 10:27:26  14.985189
1  2021-12-10 10:27:31  14.985189
2  2021-12-10 10:27:36  14.985189
3  2021-12-10 10:27:41  14.985189
4  2021-12-10 10:27:46  14.985189
..                 ...        ...
56 2021-12-10 10:32:06  14.987462
57 2021-12-10 10:32:11  14.987462
58 2021-12-10 10:32:16  14.987462
59 2021-12-10 10:32:21  14.987462
60 2021-12-10 10:32:26  14.987462

[61 rows x 2 columns]
}
{
metric_name: 'prometheus_target_interval_length_seconds'
label_config: {'instance': 'localhost:9090', 'interval': '15s', 'job': 'prometheus', 'quantile': '0.05'}
metric_values:                     ds          y
0  2021-12-10 10:27:26  14.987462
1  2021-12-10 10:27:31  14.987462
2  2021-12-10 10:27:36  14.987462
3  2021-12-10 10:27:41  14.987462
4  2021-12-10 10:27:46  14.987462
..   

the metric values are saved as df with the exact column names, ds and y, and the exact format: datetime and float!

In [24]:
tmp = metric_object_list[0]

In [25]:
tmp.metric_values

,ds,y
0,2021-12-10 10:27:26,14.985189
1,2021-12-10 10:27:31,14.985189
2,2021-12-10 10:27:36,14.985189
3,2021-12-10 10:27:41,14.985189
4,2021-12-10 10:27:46,14.985189
...,...,...
56,2021-12-10 10:32:06,14.987462
57,2021-12-10 10:32:11,14.987462
58,2021-12-10 10:32:16,14.987462
59,2021-12-10 10:32:21,14.987462


In [26]:
tmp.metric_values.dtypes

ds    datetime64[ns]
y            float64
dtype: object

also, we can manipulate the label config dict to a string that can be called in prometheus api:

In [27]:
str(tmp.label_config).replace(", '",", ").replace("': ","=").replace("{'","{")

"{instance='localhost:9090', interval='15s', job='prometheus', quantile='0.01'}"

but life isn't so easy. when the query is not raw metric but rather some expression we get error when trying to create MetricList object:

In [28]:
query = "rate(go_memstats_alloc_bytes_total[1m])"

In [29]:
query_range = prom.custom_query_range(query ,
                            start_time=start_time,
                            end_time=end_time,
                            step=step)



In [30]:
metric_object_list = MetricsList(query_range)

KeyError: '__name__'

Of course: expressions doesn't have \_\_name\_\_ property... hence it seems that MetricList is suited only to basic metrics. what about MetricRangeDataFrame()? let's check:

In [31]:
df = MetricRangeDataFrame(query_range)

In [32]:
df.head()

,instance,job,value
timestamp,,,
1639132046,localhost:9090,prometheus,19511.735419630157
1639132051,localhost:9090,prometheus,22901.322663109924
1639132056,localhost:9090,prometheus,22901.322663109924
1639132061,localhost:9090,prometheus,22901.322663109924
1639132066,localhost:9090,prometheus,25025.42222222222


so MetricRangeDataFrame can handle this cases by ommiting the \_\_name\_\_ column

let's see if it can handle expressions that output multiple time-series as in expression on summary metric:

In [33]:
query = "rate(prometheus_target_interval_length_seconds[1m])"
query_range = prom.custom_query_range(query ,
                            start_time=start_time,
                            end_time=end_time,
                            step=step)

In [34]:
len(query_range)

5

In [35]:
query_range

[{'metric': {'instance': 'localhost:9090',
   'interval': '15s',
   'job': 'prometheus',
   'quantile': '0.01'},
  'values': [[1639132046, '0'],
   [1639132051, '0'],
   [1639132056, '0'],
   [1639132061, '0'],
   [1639132066, '0'],
   [1639132071, '0'],
   [1639132076, '0'],
   [1639132081, '0'],
   [1639132086, '0'],
   [1639132091, '0'],
   [1639132096, '0'],
   [1639132101, '0'],
   [1639132106, '0'],
   [1639132111, '0'],
   [1639132116, '0'],
   [1639132121, '0'],
   [1639132126, '0'],
   [1639132131, '0'],
   [1639132136, '0'],
   [1639132141, '0'],
   [1639132146, '0'],
   [1639132151, '0'],
   [1639132156, '0'],
   [1639132161, '0'],
   [1639132166, '0'],
   [1639132171, '0'],
   [1639132176, '0'],
   [1639132181, '0'],
   [1639132186, '0'],
   [1639132191, '0'],
   [1639132196, '0'],
   [1639132201, '0'],
   [1639132206, '0'],
   [1639132211, '0'],
   [1639132216, '0'],
   [1639132221, '0'],
   [1639132226, '0'],
   [1639132231, '0'],
   [1639132236, '0'],
   [1639132241, '0'

In [42]:
metric_object_list = MetricsList(query_range)

KeyError: '__name__'

again, MetricsList failed but MetricRangeDataFrame can handle it:

In [45]:
df = MetricRangeDataFrame(query_range)
df.head()

,instance,interval,job,quantile,value
timestamp,,,,,
1639132046,localhost:9090,15s,prometheus,0.01,0
1639132051,localhost:9090,15s,prometheus,0.01,0
1639132056,localhost:9090,15s,prometheus,0.01,0
1639132061,localhost:9090,15s,prometheus,0.01,0
1639132066,localhost:9090,15s,prometheus,0.01,0


and again it ommits the \_\_name\_\_ column.

On one hand, we can just require from the user that all the queries will be specific such that they yield only one time-series. This will ease significantly all the process of make forecasts in the trainer, save them and extract them in the detector, along with the real-time data. On the other hand, maybe this is too restrictive requirement, since a lot of expresions in prometheus are natively designed to yield multiple time-series... So lets see what are the implications of supporting multiple time-series queries in our trainer-detector architecture:

Lets count the relevant processes in the trainer and the detector that need special attention to support multi-time-series queries:
1. extract the data from each query - need to use at least custom_query_range. Possibly pass the output to MetricRangeDataFrame, but not MetricsList (which not support non-metric expressions). In the general case we will get multiple metrics with unique labels values combination for each metric.
2. fit_predict - for each query need to run an inner loop and fit_predict each time-series.
3. save the forecasts - in some format... it can be csv for each individual metric with naming convention which includes the query and all the labels... can cause to more than 250 characters... not so good... maybe dump all the query_range data or the whole dataframe and the filenames will be serial number or shortened unique name (as done in Taboola).
4. getting real time-data - to get the labels of each time-series we need to call custom_query_range first to get the real-time data (last 10 minutes).
5. get forecasts - after getting all the labels combinations we need to extract their corresponding forecasts. for this we can compare the custom_query_range results above to the saved forecast data and use the data with the corresponding matched labels.


So supporting multiple time-series queries seems to be duable but need quite complex handling, so we can keep it as future feature and for now we restrict the queries to yield single time-series.

BUT! there is simple solution to supporting multi-time-series case and it calls RULES. The solution is to prohibit using of custom queries and require the user to use only basic metrics or recorded rules. Since recorded rules has name than we can use MetricsList. Moreover, since we no longer use custom queries we will use get_metric_range_data() instead. lets try it:

In [46]:
start_time = parse_datetime("2021-12-08 23:30")
end_time = parse_datetime("2021-12-08 23:35")

metric = "prometheus_target_interval_length_seconds"
metric_data = prom.get_metric_range_data(metric_name=metric ,
                            start_time=start_time,
                            end_time=end_time)
metric_object_list = MetricsList(metric_data)

C:\Users\Yair\miniconda3\lib\site-packages\dateparser\date_parser.py:35: PytzUsageWarning: The localize method is no longer necessary, as this time zone supports the fold attribute (PEP 495). For more details on migrating to a PEP 495-compliant implementation, see https://pytz-deprecation-shim.readthedocs.io/en/latest/migration.html
  date_obj = stz.localize(date_obj)


In [47]:
for item in metric_object_list:
    print(item)

{
metric_name: 'prometheus_target_interval_length_seconds'
label_config: {'instance': 'localhost:9090', 'interval': '15s', 'job': 'prometheus', 'quantile': '0.01'}
metric_values:                               ds          y
0  2021-12-08 21:30:12.989000082  14.989236
1  2021-12-08 21:30:27.983999968  14.989236
2  2021-12-08 21:30:42.987999916  14.989236
3  2021-12-08 21:30:57.977999926  14.989236
4  2021-12-08 21:31:12.976999998  14.989236
5  2021-12-08 21:31:27.980000019  14.989236
6  2021-12-08 21:31:42.977999926  14.989236
7  2021-12-08 21:31:57.974999905  14.989236
8  2021-12-08 21:32:12.989000082  14.989236
9  2021-12-08 21:32:27.986999989  14.989236
10 2021-12-08 21:32:42.986000061  14.989236
11 2021-12-08 21:32:57.976999998  14.989236
12 2021-12-08 21:33:12.980999947  14.990070
13 2021-12-08 21:33:27.986000061  14.990070
14 2021-12-08 21:33:42.987999916  14.990070
15 2021-12-08 21:33:57.987999916  14.990070
16 2021-12-08 21:34:12.986000061  14.990070
17 2021-12-08 21:34:27.986000

the difference here (w.r.t query) is that we are restricted to the scrape interval defined for the metric or the rule. In addition, the timestemps are not integer multiples of the scraping interval and we need to treat it in purpose to align the timestamps between the trainer and the detector:

In [48]:
df = metric_object_list[0].metric_values
df

,ds,y
0,2021-12-08 21:30:12.989000082,14.989236
1,2021-12-08 21:30:27.983999968,14.989236
2,2021-12-08 21:30:42.987999916,14.989236
3,2021-12-08 21:30:57.977999926,14.989236
4,2021-12-08 21:31:12.976999998,14.989236
5,2021-12-08 21:31:27.980000019,14.989236
6,2021-12-08 21:31:42.977999926,14.989236
7,2021-12-08 21:31:57.974999905,14.989236
8,2021-12-08 21:32:12.989000082,14.989236
9,2021-12-08 21:32:27.986999989,14.989236


In [49]:
df.resample('15s',label='right', on='ds').mean().reset_index()

,ds,y
0,2021-12-08 21:30:15,14.989236
1,2021-12-08 21:30:30,14.989236
2,2021-12-08 21:30:45,14.989236
3,2021-12-08 21:31:00,14.989236
4,2021-12-08 21:31:15,14.989236
5,2021-12-08 21:31:30,14.989236
6,2021-12-08 21:31:45,14.989236
7,2021-12-08 21:32:00,14.989236
8,2021-12-08 21:32:15,14.989236
9,2021-12-08 21:32:30,14.989236


explore store_locally option of get_metric_range_data:

In [50]:
prom.get_metric_range_data(metric_name=metric ,
                            start_time=start_time,
                            end_time=end_time, store_locally=True)

NotADirectoryError: [WinError 267] The directory name is invalid: './metrics/localhost:9090'

because of the ':' it doesn't supported in windows... so we need to do it manually:

In [52]:
metric_data

[{'metric': {'__name__': 'prometheus_target_interval_length_seconds',
   'instance': 'localhost:9090',
   'interval': '15s',
   'job': 'prometheus',
   'quantile': '0.01'},
  'values': [[1638999012.989, '14.9892357'],
   [1638999027.984, '14.9892357'],
   [1638999042.988, '14.9892357'],
   [1638999057.978, '14.9892357'],
   [1638999072.977, '14.9892357'],
   [1638999087.98, '14.9892357'],
   [1638999102.978, '14.9892357'],
   [1638999117.975, '14.9892357'],
   [1638999132.989, '14.9892357'],
   [1638999147.987, '14.9892357'],
   [1638999162.986, '14.9892357'],
   [1638999177.977, '14.9892357'],
   [1638999192.981, '14.9900703'],
   [1638999207.986, '14.9900703'],
   [1638999222.988, '14.9900703'],
   [1638999237.988, '14.9900703'],
   [1638999252.986, '14.9900703'],
   [1638999267.986, '14.9900703'],
   [1638999282.975, '14.9888206'],
   [1638999297.979, '14.9888206']]},
 {'metric': {'__name__': 'prometheus_target_interval_length_seconds',
   'instance': 'localhost:9090',
   'interval'

but if we will do it manually we want to save the resampled data with integer multiples timestamps. On the other hand the resampling is done on the MetricsList object, becasue only there the data format is pandas DataFrame..
So there are 2 options:
1. Use get_metric_range_data, wrap the result with MetricList and save the MetricsList object in binary format (pickle)
2. although we use only metrics and rules we still can use the custom_query_range and benefit from the implicit resampling of this method (and save the results in pickle format after wrapping it with MetricList object)

Since option 2 eliminates the explicit resampling we will use it.

The reason for using pickle and not json is because we wrap the results in MetricList object which cannot be serialized by json. Also, the MetricList object itself contains pandas dataframe (in metric_values) so if we really want to use json we need double conversion: first, from MetricList to dict and then dataframe to list of lists (or touples), but why to bother?

In [93]:
start_time = parse_datetime("2021-12-05 08:00")
end_time = parse_datetime("2021-12-05 20:00")

metric = "prometheus_target_interval_length_seconds"
metric_data = prom.custom_query_range(metric ,
                            start_time=start_time,
                            end_time=end_time, step='15s')
metric_object_list = MetricsList(metric_data)

C:\Users\Yair\miniconda3\lib\site-packages\dateparser\date_parser.py:35: PytzUsageWarning: The localize method is no longer necessary, as this time zone supports the fold attribute (PEP 495). For more details on migrating to a PEP 495-compliant implementation, see https://pytz-deprecation-shim.readthedocs.io/en/latest/migration.html
  date_obj = stz.localize(date_obj)


In [94]:
tmp = metric_object_list[0]

In [95]:
tmp.metric_values

,ds,y
0,2021-12-05 06:00:00,4.985374
1,2021-12-05 06:00:15,4.985374
2,2021-12-05 06:00:30,4.985374
3,2021-12-05 06:00:45,4.985374
4,2021-12-05 06:01:00,4.985374
...,...,...
2610,2021-12-05 17:59:00,4.982201
2611,2021-12-05 17:59:15,4.982201
2612,2021-12-05 17:59:30,4.985147
2613,2021-12-05 17:59:45,4.983872


In [3]:
with open('forecasts/prometheus_tsdb_head_chunks.pkl', 'rb') as inp:
    tmp = pickle.load(inp)

In [4]:
tmp

In [5]:
tmp[0].metric_values

,ds,yhat,yhat_lower,yhat_upper
1801,2022-02-25 11:30:02,1161.035180,960.925292,1353.450588
1802,2022-02-25 11:30:04,1164.583912,980.908387,1370.061980
1803,2022-02-25 11:30:06,1168.132644,930.234567,1375.829399
1804,2022-02-25 11:30:08,1171.681377,965.446296,1388.788865
1805,2022-02-25 11:30:10,1175.230109,1007.715836,1373.538931
...,...,...,...,...
2036,2022-02-25 11:37:52,1994.987235,1600.584018,2412.370239
2037,2022-02-25 11:37:54,1998.535967,1551.283574,2395.034365
2038,2022-02-25 11:37:56,2002.084700,1574.840268,2390.355932
2039,2022-02-25 11:37:58,2005.633432,1554.646500,2407.454458
